# Mafunzo ya Phi-4 Mini ONNX Parallel Function Calling

Notibuku hii inaonyesha jinsi ya kutumia Phi-4 Mini na ONNX Runtime GenAI kwa ajili ya kuita kazi kwa njia sambamba. Kuita kazi kunaruhusu modeli kuitisha zana za nje na API kwa akili kulingana na maombi ya mtumiaji.

## Muhtasari

Katika mafunzo haya, utajifunza jinsi ya:
- Kuweka Phi-4 Mini na ONNX Runtime GenAI
- Kufafanua miundo ya kazi kwa ajili ya kuhifadhi tiketi za ndege na hoteli
- Kutumia kizazi kinachoongozwa na sarufi ya Lark kwa matokeo yaliyo na muundo
- Kutekeleza miito ya kazi sambamba kwa hali ngumu za uhifadhi wa safari

## Mahitaji ya Awali

Kabla ya kuendesha notibuku hii, hakikisha una:
- Kupakua modeli ya Phi-4 Mini ONNX
- Kusakinisha kifurushi cha `onnxruntime-genai`
- Uelewa wa msingi wa dhana za kuita kazi


## Hatua ya 1: Leta Maktaba Zinazohitajika

Kwanza, tutaingiza maktaba muhimu kwa utekelezaji wa kupiga kazi zetu.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## Hatua ya 2: Usanidi wa Muundo na Uwekaji wa Vigezo

Sasa tutaweka na kusanidi mfano wa Phi-4 Mini ONNX. Hakikisha unabadilisha njia na kutumia saraka halisi ya mfano wako.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## Hatua ya 3: Sanidi Vigezo vya Uzalishaji

Sanidi vigezo vya uzalishaji ili kudhibiti tabia ya matokeo ya modeli. Mipangilio hii inahakikisha majibu ya uhakika na yenye mwelekeo kwa kuita kazi.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Hatua ya 4: Eleza Kazi Zinazopatikana

Hapa tunaeleza kazi ambazo msaidizi wetu wa AI anaweza kutumia. Katika mfano huu, tuna kazi mbili zinazohusiana na safari:
1. **booking_flight_tickets**: Kwa kuhifadhi tiketi za ndege kati ya viwanja vya ndege
2. **booking_hotels**: Kwa kuhifadhi malazi ya hoteli

Ufafanuzi wa kazi unafuata muundo wa OpenAI wa mpangilio wa kuita kazi.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Hatua ya 5: Kazi Msaidizi za Kutengeneza Sarufi

Kazi hizi msaidizi hubadilisha ufafanuzi wa kazi zetu kuwa muundo wa sarufi ya Lark, ambayo hutumiwa na ONNX Runtime GenAI kwa kizazi kinachoongozwa. Hii inahakikisha kwamba modeli inatoa miito ya kazi sahihi katika muundo wa JSON unaofaa.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## Hatua ya 6: Jaribu Uundaji wa Sarufi

Hebu tujaribu kazi zetu za uundaji wa sarufi ili kuona jinsi zinavyobadilisha ufafanuzi wa zana zetu kuwa muundo unaofaa.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## Hatua ya 7: Andaa Kidokezo cha Mfumo na Kizalishi

Sasa tutaandaa kidokezo cha mfumo kinachomweleza mfano kuhusu zana zinazopatikana na kusanidi kizalishi kwa vigezo vya kizazi kilichoelekezwa.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## Hatua ya 8: Anzisha Jenereta kwa Uzalishaji Unaongoza

Sasa tutaunda jenereta kwa kutumia vigezo vyetu vilivyosanidiwa na kutumia sarufi ya Lark kwa uzalishaji unaoongozwa.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## Hatua ya 9: Jaribu Kuita Kazi Sambamba

Sasa hebu tujaribu usanidi wetu na hali changamano ya kuhifadhi safari inayohitaji kuita kazi nyingi.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Tengeneza Simu za Kazi

Mfumo sasa utazalisha simu za kazi zilizopangwa kulingana na ombi la mtumiaji. Shukrani kwa kizazi kilichoelekezwa, matokeo yatakuwa katika muundo halali wa JSON ambao unaweza kutekelezwa moja kwa moja.

**Matokeo Yanayotarajiwa**: Mfumo unapaswa kuzalisha simu za kazi kwa:
1. `booking_flight_tickets` na maelezo ya Beijing (PEK) hadi Paris (CDG)
2. `booking_hotels` na maelezo ya malazi ya Paris

Endesha seli hapa chini ili kuona kizazi cha moja kwa moja:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Hitimisho

🎉 **Hongera!** Umefanikiwa kutekeleza kupiga simu za kazi sambamba kwa kutumia Phi-4 Mini na ONNX Runtime GenAI.

### Ulichojifunza:

1. **Usanidi wa Modeli**: Jinsi ya kusanidi Phi-4 Mini na ONNX Runtime GenAI  
2. **Ufafanuzi wa Kazi**: Jinsi ya kufafanua miundo ya kazi kwa kupiga simu za AI  
3. **Uzalishaji Unaosimamiwa**: Jinsi ya kutumia sarufi ya Lark kwa uzalishaji wa matokeo yaliyojengwa  
4. **Simu za Kazi Sambamba**: Jinsi ya kushughulikia maombi magumu yanayohitaji simu nyingi za kazi  

### Faida Muhimu:

- ✅ **Matokeo Yaliyojengwa**: Uzalishaji unaosimamiwa unahakikisha simu za kazi za JSON halali  
- ✅ **Usindikaji Sambamba**: Kushughulikia simu nyingi za kazi katika ombi moja  
- ✅ **Utendaji wa Juu**: ONNX Runtime hutoa utabiri ulioboreshwa  
- ✅ **Muundo Rahisi**: Rahisi kuongeza au kurekebisha ufafanuzi wa kazi  

### Rasilimali:

- [Phi-4 Mini Documentation](https://huggingface.co/microsoft/Phi-4-mini-onnx)  
- [ONNX Runtime GenAI Documentation](https://onnxruntime.ai/docs/genai/)  
- [Function Calling Best Practices](https://platform.openai.com/docs/guides/function-calling)  



---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya tafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kwa usahihi, tafadhali fahamu kuwa tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati ya asili katika lugha yake ya awali inapaswa kuzingatiwa kama chanzo cha mamlaka. Kwa taarifa muhimu, inashauriwa kutumia tafsiri ya kitaalamu ya binadamu. Hatutawajibika kwa maelewano mabaya au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
